In [ ]:
import numpy as np;
import scipy.signal as signal
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
def firws(m, f , w , t = None):
    """
    Designs windowed sinc type I linear phase FIR filter.
    Parameters:        
        m: filter order.
        f: cutoff frequency/ies (-6 dB;pi rad / sample).
        w: vector of length m + 1 defining window. 
        t: 'high' for highpass, 'stop' for bandstop filter. {default low-/bandpass}
    Returns:
        b: numpy.ndarray
            filter coefficients 
    """
    f = np.squeeze(f)
    f = f / 2; 
    w = np.squeeze(w)
    if (f.ndim == 0): #low pass
        b = fkernel(m, f, w)
    else:
        b = fkernel(m, f[0], w) #band
    
    if (f.ndim == 0) and (t == 'high'):
        b = fspecinv(b)
    elif (f.size == 2):
        b = b + fspecinv(fkernel(m, f[1], w)) #reject
        if t == None or (t != 'stop'):
            b = fspecinv(b) #bandpass        
    return b

In [ ]:
# Compute filter kernel
def fkernel(m, f, w):
    m = np.arange(-m/2, (m/2)+1)
    b = np.zeros((m.shape[0]))
    b[m==0] = 2*np.pi*f # No division by zero
    b[m!=0] = np.sin(2*np.pi*f*m[m!=0]) / m[m!=0] # Sinc
    b = b * w # Windowing
    b = b / np.sum(b) # Normalization to unity gain at DC
    return b

In [ ]:
## Spectral inversion
def fspecinv(b):
    b = -b
    b[int((b.shape[0]-1)/2)] = b[int((b.shape[0]-1)/2)]+1
    return b

In [ ]:
#heuristics for band management
def filter_design(srate, locutoff = 0, hicutoff = 0, revfilt = 0):
    #Constants
    TRANSWIDTHRATIO = 0.25;
    fNyquist = srate/2;  
    
    #The prototipical filter is the low-pass, we design a low pass and transform it
    if hicutoff == 0: #Convert highpass to inverted lowpass
        hicutoff = locutoff
        locutoff = 0
        revfilt = 1 #invert the logic for low-pass to high-pass and for
                    #band-pass to notch
    if locutoff > 0 and hicutoff > 0:
        edgeArray = np.array([locutoff , hicutoff])
    else:
        edgeArray = np.array([hicutoff]);
    
    #Not negative frequencies and not frequencies above Nyquist
    if np.any(edgeArray<0) or np.any(edgeArray >= fNyquist):
        print('Cutoff frequency out of range')
        return False  
    
    # Max stop-band width
    maxBWArray = edgeArray.copy() # Band-/highpass
    if revfilt == 0: # Band-/lowpass
        maxBWArray[-1] = fNyquist - edgeArray[-1];
    elif len(edgeArray) == 2: # Bandstop
        maxBWArray = np.diff(edgeArray) / 2;
    maxDf = np.min(maxBWArray);
    
    # Default filter order heuristic
    if revfilt == 1: # Highpass and bandstop
        df = np.min([np.max([maxDf * TRANSWIDTHRATIO, 2]) , maxDf]);
    else: # Lowpass and bandpass
        df = np.min([np.max([edgeArray[0] * TRANSWIDTHRATIO, 2]) , maxDf]);
    
    print(df)
    
    filtorder = 3.3 / (df / srate); # Hamming window
    filtorder = np.ceil(filtorder / 2) * 2; # Filter order must be even.
    
    # Passband edge to cutoff (transition band center; -6 dB)
    dfArray = [[df, [-df, df]] , [-df, [df, -df]]];
    cutoffArray = edgeArray + np.array(dfArray[revfilt][len(edgeArray) - 1]) / 2;
    print('pop_eegfiltnew() - cutoff frequency(ies) (-6 dB): '+str(cutoffArray)+' Hz\n');
    # Window
    winArray = signal.hamming(int(filtorder) + 1);
    # Filter coefficients
    if revfilt == 1:
        filterTypeArray = ['high', 'stop'];
        b = firws(filtorder, cutoffArray / fNyquist, winArray, filterTypeArray[len(edgeArray) - 1]);
    else:
        b = firws(filtorder, cutoffArray / fNyquist, winArray);

    return filtorder, b; 

In [ ]:
def mfreqz(b,a,order,nyq_rate = 1):
    
    """
    Plot the impulse response of the filter in the frequency domain

    Parameters:
        
        b: numerator values of the transfer function (coefficients of the filter)
        a: denominator values of the transfer function (coefficients of the filter)
        
        order: order of the filter 
                
        nyq_rate = nyquist frequency
    """
    
    w,h = signal.freqz(b,a);
    h_dB = 20 * np.log10 (abs(h));
    
    plt.figure();
    plt.subplot(311);
    plt.plot((w/max(w))*nyq_rate,abs(h));
    plt.ylabel('Magnitude');
    plt.xlabel(r'Normalized Frequency (x$\pi$rad/sample)');
    plt.title(r'Frequency response. Order: ' + str(order));
    [xmin, xmax, ymin, ymax] = plt.axis();
    
    #plt.xlim((40,60))
    
    plt.grid(True);
    
    plt.subplot(312);
    plt.plot((w/max(w))*nyq_rate,h_dB);
    plt.ylabel('Magnitude (db)');
    plt.xlabel(r'Normalized Frequency (x$\pi$rad/sample)');
    plt.title(r'Frequency response. Order: ' + str(order));
    plt.grid(True)
    plt.grid(True)
    
    
    plt.subplot(313);
    h_Phase = np.unwrap(np.arctan2(np.imag(h),np.real(h)));
    plt.plot((w/max(w))*nyq_rate,h_Phase);
    plt.ylabel('Phase (radians)');
    plt.xlabel(r'Normalized Frequency (x$\pi$rad/sample)');
    plt.title(r'Phase response. Order: ' + str(order));
    plt.subplots_adjust(hspace=0.5);
    plt.grid(True)
    plt.show()

In [ ]:
#low pass filter 50Hz with order = 265 (for a transition band of 25%) and hamming window
#fs = 1000
b = fkernel(264, 50/1000, signal.hamming(int(264) + 1))
plt.stem(b)

mfreqz(b,1,264, 500);

In [ ]:
print(b)

In [ ]:
#high pass filter 5Hz with order = 265 (for a transition band of 25%) and hamming window
from scipy import signal

b = fkernel(264, 5/1000, signal.hamming(int(264) + 1))
plt.subplot(3,1,1)
plt.plot(b)

delta = signal.unit_impulse(265,133)
plt.subplot(3,1,2)
plt.plot(delta)

bhpf = delta - b
plt.subplot(3,1,3)
plt.plot(bhpf)

mfreqz(bhpf,1,264, 500);

In [ ]:
fs = 250;
#design
order, lowpass = filter_design(fs, locutoff = 0, hicutoff = 50, revfilt = 0);
plt.plot(lowpass)
#plot
mfreqz(lowpass,1,order, fs/2);

In [ ]:
order, highpass = filter_design(fs, locutoff = 4, hicutoff = 0, revfilt = 1);
plt.plot(highpass)
#plot
mfreqz(highpass,1,order, fs/2);

In [ ]:
order, bandpass = filter_design(fs, locutoff = 4, hicutoff = 50, revfilt = 0);
#plot
mfreqz(bandpass,1,order, fs/2);

In [ ]:
order, notch = filter_design(fs, locutoff = 50, hicutoff = 70, revfilt = 1);
#plot
mfreqz(notch,1,order, fs/2);

In [ ]:
#%%load signal and apply filter
import scipy.io as sio;
#carga de los datos
mat_contents = sio.loadmat('/content/drive/Shareddrives/biosenales_sistemas/filtrado/senal_anestesia.mat')
#los datos se cargan como un diccionario, se puede evaluar los campos que contiene
print("Los campos cargados son: " + str(mat_contents.keys()));
#la senal esta en el campo data
senal_org = mat_contents['data'];
senal_org = senal_org[0,:];

senal_filtrada_pasaaltas = signal.filtfilt(highpass, 1, senal_org);
senal_filtrada_pasabajas = signal.filtfilt(lowpass, 1, senal_org);
senal_filtrada_pasabanda = signal.filtfilt(bandpass, 1, senal_org);
senal_filtrada_rechazabanda = signal.filtfilt(notch, 1, senal_org);

plt.subplot(2,2,1);
plt.plot(senal_org[0:250]);
plt.plot(senal_filtrada_pasaaltas[0:250]);
plt.title('Pasa-altas');
plt.subplot(2,2,2);
plt.plot(senal_org[0:250]);
plt.plot(senal_filtrada_pasabajas[0:250]);
plt.title('Pasa-bajas');
plt.subplot(2,2,3);
plt.plot(senal_org[0:250]);
plt.plot(senal_filtrada_pasabanda[0:250]);
plt.title('Pasa-banda');
plt.subplot(2,2,4);
plt.plot(senal_org[0:250]);
plt.plot(senal_filtrada_rechazabanda[0:250]);
plt.title('Rechaza-banda');

plt.show();

In [ ]:
import scipy.signal as pds

f, Pxx = pds.welch(senal_org, fs, 'hann', fs*2, fs)
plt.title('Espectro original');
plt.plot(f, Pxx);
plt.show()

In [ ]:
f, Pxx = pds.welch(senal_filtrada_pasaaltas, fs, 'hann', fs*2, fs)
plt.title('Espectro original - pasaaltas');
plt.plot(f, Pxx);
plt.show()

In [ ]:
f, Pxx = pds.welch(senal_filtrada_pasabajas, fs, 'hann', fs*2, fs)
plt.title('Espectro original - pasabajas');
plt.stem(f, Pxx);
plt.show()

In [ ]:
f, Pxx = pds.welch(senal_filtrada_rechazabanda, fs, 'hann', fs*2, fs)
plt.title('Espectro original - rechazabandas');
plt.plot(f, Pxx);
plt.show()

In [ ]:
f, Pxx = pds.welch(senal_filtrada_pasabanda, fs, 'hann', fs*2, fs)
plt.title('Espectro original - pasabanda');
plt.stem(f, Pxx);
plt.show()

In [ ]:
f, Pxx = pds.welch(senal_filtrada_pasabajas, fs, 'hann', fs*2, fs)
plt.subplot(2,2,1);
plt.title('Espectro original - pasaaltas');
plt.xlim([0,50]);
plt.plot(f, Pxx);

f, Pxx = pds.welch(senal_filtrada_pasabajas, fs, 'hann', fs*4, fs*2)
plt.subplot(2,2,2);
plt.title('Espectro original - pasaaltas');
plt.xlim([0,50]);
plt.plot(f, Pxx);

f, Pxx = pds.welch(senal_filtrada_pasabajas, fs, 'hann', fs*8, fs*4)
plt.subplot(2,2,3);
plt.title('Espectro original - pasaaltas');
plt.xlim([0,50]);
plt.plot(f, Pxx);

f, Pxx = pds.welch(senal_filtrada_pasabajas, fs, 'hann', fs*10, fs*5)
plt.subplot(2,2,4);
plt.title('Espectro original - pasaaltas');
plt.plot(f, Pxx);
plt.xlim([0,50]);
plt.show()